In [1]:
param_url = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=ignition+coil+blue&_sacat=0&rt=nc&LH_PrefLoc=1'

In [2]:
dict_param = {}
list_param = param_url.split('?')[1].split('&') if '?' in param_url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param

{'_from': 'R40',
 '_nkw': 'ignition+coil+blue',
 '_sacat': '0',
 'rt': 'nc',
 'LH_PrefLoc': '1'}

In [3]:
dict_param.pop('_pgn', '')
dict_param['orig_cvip'] = 'true'
dict_param['_ipg'] = '240'

if param_url.startswith('https://www.ebay.com/'):
    dict_param['_ul'] = 'US'
    dict_param['_stpos'] = '91710'
elif param_url.startswith('https://www.ebay.de/'):
    dict_param['_ul'] = 'DE'
    dict_param['_stpos'] = '10115'
elif param_url.startswith('https://www.ebay.co.uk/'):
    dict_param['_ul'] = 'GB'
    dict_param['_stpos'] = 'BS81QU'
elif param_url.startswith('https://www.ebay.com.au/'):
    dict_param['_ul'] = 'AU'
    dict_param['_stpos'] = '2019'

dict_param

{'_from': 'R40',
 '_nkw': 'ignition+coil+blue',
 '_sacat': '0',
 'rt': 'nc',
 'LH_PrefLoc': '1',
 'orig_cvip': 'true',
 '_ipg': '240',
 '_ul': 'US',
 '_stpos': '91710'}

In [4]:
param_url = param_url.split('?')[0] + '?'
for key, value in dict_param.items():
    param_url += f'{key}={value}&'

param_url

'https://www.ebay.com/sch/i.html?_from=R40&_nkw=ignition+coil+blue&_sacat=0&rt=nc&LH_PrefLoc=1&orig_cvip=true&_ipg=240&_ul=US&_stpos=91710&'

In [5]:
import requests

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

import json

from bs4 import BeautifulSoup
from lxml import etree

list_item_number, page = [], 1
while True:
    b = 0
    while True:
        b += 1
        print(b, page)
        try:
            resp = requests.get(f'''{param_url}_pgn={page}''',
                                headers=get_header(ua=False),
                                proxies=get_proxy(),
                                timeout=(10, 10))

            if resp.status_code == 200:
                break
        except:
            continue

    # = = = = = = = = = = = = = = =

    soup = BeautifulSoup(resp.text, 'lxml')
    html = etree.HTML(str(soup))

    # = = = = = = = = = = = = = = =

    json_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0].strip() + ']}]}')
    
    # = = = = = = = = = = = = = = =

    list_item_number += [w[2]['model']['action']['params']['itemId'].strip() if type(w[2]) == dict and 'model' in w[2] and 'action' in w[2]['model'] and 'params' in w[2]['model']['action'] and 'itemId' in w[2]['model']['action']['params'] else '' for w in json_['o']['w']]
    while '' in list_item_number:
        list_item_number.remove('')

    list_item_number = sorted(list(set(list_item_number)), key=list_item_number.index)
        
    # = = = = = = = = = = = = = = =

    page += 1

    # = = = = = = = = = = = = = = =

    if not html.xpath('//a[@type="next"]'):
        break
    
resp

1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
2 13
3 13
4 13
5 13
6 13
7 13
8 13
9 13
10 13
11 13
12 13
13 13
14 13
15 13
16 13
17 13
18 13
19 13
20 13
21 13
22 13
23 13
24 13
25 13
26 13
27 13
28 13
29 13
30 13
31 13
32 13
33 13
34 13
35 13
36 13
37 13
38 13
39 13
40 13
41 13
42 13
43 13
44 13
45 13
46 13
47 13
48 13
49 13
50 13
51 13
52 13
53 13
54 13
55 13
56 13
57 13
58 13
59 13
60 13
61 13
62 13
63 13
64 13
65 13
66 13
67 13
68 13
69 13
70 13
71 13
72 13
73 13
74 13
75 13
76 13
77 13
78 13
79 13
80 13
81 13
82 13
83 13
84 13
85 13
86 13
87 13
88 13
89 13
90 13
91 13
92 13
93 13
94 13
95 13
96 13
97 13
98 13
99 13
100 13
101 13
102 13
103 13
104 13
105 13
106 13
107 13
108 13
109 13
110 13
111 13
112 13
113 13
114 13
115 13
116 13
117 13
118 13
119 13
120 13
121 13
122 13
123 13
124 13
125 13
126 13
127 13
128 13
129 13
130 13
131 13
132 13
133 13
134 13
135 13
1 14


<Response [200]>

In [ ]:
len(list_item_number) == 0

In [6]:
import pandas as pd

output = pd.DataFrame({'No': [i+1 for i in range(len(list_item_number))],
                       'Item Number': list_item_number})

output['No'] = [_+1 for _ in range(len(output))]
if param_url.startswith('https://www.ebay.com/'):
    output['Url'] = [f'''https://www.ebay.com/itm/{_}?_ul=US&_stpos=91710&orig_cvip=true''' for _ in output['Item Number'].tolist()]
elif param_url.startswith('https://www.ebay.de/'):
    output['Url'] = [f'''https://www.ebay.de/itm/{_}?_ul=DE&_stpos=10115&orig_cvip=true''' for _ in output['Item Number'].tolist()]
elif param_url.startswith('https://www.ebay.co.uk/'):
    output['Url'] = [f'''https://www.ebay.co.uk/itm/{_}?_ul=GB&_stpos=BS81QU&orig_cvip=true''' for _ in output['Item Number'].tolist()]
elif param_url.startswith('https://www.ebay.com.au/'):
    output['Url'] = [f'''https://www.ebay.com.au/itm/{_}?_ul=AU&_stpos=2019&orig_cvip=true''' for _ in output['Item Number'].tolist()]

output.to_excel('./1.menu.xlsx', index=False)

output

,No,Item Number,Url
0,1,335163287068,https://www.ebay.com/itm/335163287068?_ul=US&_...
1,2,404195442365,https://www.ebay.com/itm/404195442365?_ul=US&_...
2,3,335160304884,https://www.ebay.com/itm/335160304884?_ul=US&_...
3,4,335423079192,https://www.ebay.com/itm/335423079192?_ul=US&_...
4,5,384400617283,https://www.ebay.com/itm/384400617283?_ul=US&_...
...,...,...,...
3315,3316,181847780007,https://www.ebay.com/itm/181847780007?_ul=US&_...
3316,3317,191679949516,https://www.ebay.com/itm/191679949516?_ul=US&_...
3317,3318,116207375058,https://www.ebay.com/itm/116207375058?_ul=US&_...
3318,3319,126515187518,https://www.ebay.com/itm/126515187518?_ul=US&_...
